# Lab: Amazon SageMaker에서 자체 커스텀 컨테이너 가져오기

<div class="alert alert-block alert-info">
⚠️ 이 노트북을 실행하기 위해서는 SageMaker Studio Domain에서 Docker가 활성화되어 있는지 확인하세요. AWS에서 진행하는 이벤트에서 이 노트북을 실행하는 경우, <b>이 부분을 건너뛰어도 됩니다</b>. 직접 SageMaker Studio Domain을 프로비저닝한 경우, <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html#studio-updated-local-enable">여기를 참조하여</a> AWS CLI를 통해 기존 SageMaker Studio Domain에서 Docker를 활성화하는 방법을 확인하세요. 이 명령을 실행한 후, <b>변경 사항을 적용하려면 JupyterApp을 재시작해야 합니다</b>.
</div>

<div class="alert alert-block alert-info">
⚠️ 이 노트북과 호환되는 것으로 알려진 최신 SageMaker Distribution 이미지 버전은 <code>3.1.0</code>입니다. 다른 버전에서 문제가 발생하면 버전 <code>3.1.0</code>으로 다운그레이드하세요. <b>이를 위해서는 JupyterApp을 중지하고, SageMaker Distribution 이미지를 <code>3.1.0</code>으로 다운그레이드한 다음, 변경 사항을 적용하기 위해 JupyterLabApp을 재시작해야 합니다</b>.</div>

## 개요

### 배경
여기서는 환경과 코드를 패키징하는 도커 컨테이너를 가져오는 방법을 보여드리겠습니다. 널리 사용되는 [scikit-learn](http://scikit-learn.org/stable/) 머신 러닝 패키지의 [decision tree](http://scikit-learn.org/stable/modules/tree.html) 알고리즘을 선보입니다. 이 예제는 의도적으로 상당히 간단하게 만들었습니다. 그 이유는 여러분이 자신의 컨테이너에 추가하고자 하는 주변 구조를 보여주는 것이 목적이기 때문입니다. 이를 통해 Amazon SageMaker에서 훈련 및 호스팅을 할 수 있습니다.


### 상위 수준 개요

다음 다이어그램은 일반적으로 Amazon SageMaker로 모델을 훈련하고 배포하는 방법을 보여줍니다:

<div>
<img src="https://docs.aws.amazon.com/sagemaker/latest/dg/images/sagemaker-architecture.png" width="900"/>
</div>

SageMaker로 표시된 영역은 SageMaker의 두 가지 구성 요소인 모델 훈련과 모델 배포를 강조합니다. [EC2 container registry](https://aws.amazon.com/ecr/)로 표시된 영역은 Docker 컨테이너 이미지를 저장, 관리 및 배포하는 곳입니다. 훈련 데이터와 모델 아티팩트는 S3 버킷에 저장됩니다.

이 실습에서는 간단하게 하기 위해 모델 훈련과 호스팅 모두에 단일 이미지를 사용합니다. 때로는 훈련과 호스팅에 서로 다른 요구 사항이 있기 때문에 별도의 이미지를 원할 수도 있습니다.

주요 단계는 다음과 같습니다:
1. **컨테이너 빌드** - 컨테이너의 다양한 구성 요소를 살펴보고 도커 파일을 검사합니다. 그런 다음 컨테이너를 빌드하고 ECR에 푸시합니다.
2. **설정 및 데이터 업로드** - 컨테이너가 빌드되고 등록되면 SageMaker를 준비하고 데이터를 S3에 업로드합니다.
3. **모델 훈련** - SageMaker Python SDK를 사용하여 훈련 작업을 생성합니다. S3에서 데이터를 가져와 우리가 빌드한 컨테이너를 사용합니다.
4. **모델 배포** - 훈련이 완료되면 SageMaker Python SDK를 사용하여 모델을 HTTP 엔드포인트에 배포합니다.
5. **추론 실행** - 모델을 테스트하기 위해 예측을 실행합니다.
6. **정리**

## 컨테이너 구축하기
[Docker](https://aws.amazon.com/docker/#:~:text=Docker%20is%20a%20software%20platform,test%2C%20and%20deploy%20applications%20quickly.&text=Running%20Docker%20on%20AWS%20provides,distributed%20applications%20at%20any%20scale.)는 소프트웨어를 [컨테이너](https://aws.amazon.com/containers/)라고 불리는 표준화된 단위로 패키징합니다. 이 컨테이너에는 소프트웨어 실행에 필요한 라이브러리, 시스템 도구, 코드 및 런타임 등 모든 것이 포함되어 있습니다. Docker를 사용하면 어떤 환경에서든 애플리케이션을 빠르게 배포하고 확장할 수 있으며, 코드가 확실히 실행될 것이라는 것을 알 수 있습니다.

Amazon SageMaker는 Docker를 사용하여 사용자가 임의의 알고리즘을 훈련하고 배포할 수 있게 합니다. [SageMaker에서 Docker 컨테이너를 사용하는 방법](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers.html)에 대한 자세한 내용을 확인하세요.

### 컨테이너 디렉토리 살펴보기
[이 GitHub 저장소](https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/scikit_bring_your_own)에서 사용 중인 샘플 컨테이너의 소스 코드를 찾을 수 있습니다.

컨테이너 디렉토리에는 SageMaker용으로 패키징해야 하는 모든 구성 요소가 포함되어 있습니다.

```
.
|-- Dockerfile
|-- build_and_push.sh
|-- local_test
`-- decision_trees
    |-- nginx.conf
    |-- predictor.py
    |-- serve
    |-- train
    `-- wsgi.py
```

각각에 대해 차례대로 살펴보겠습니다:

- `Dockerfile`은 Docker 컨테이너 이미지를 빌드하는 방법을 설명합니다. 자세한 내용은 아래에 있습니다.
- `build_and_push.sh`는 Dockerfile을 사용하여 컨테이너 이미지를 빌드한 다음 ECR에 푸시하는 스크립트입니다. 이 노트북의 뒷부분에서 명령어를 직접 호출할 것이지만, 자신의 알고리즘을 위해 스크립트를 복사하여 실행할 수 있습니다.
- `local_test`는 Amazon SageMaker 노트북 인스턴스를 포함하여 Docker를 실행할 수 있는 모든 컴퓨터에서 새 컨테이너를 테스트하는 방법을 보여주는 디렉토리입니다. 이 방법을 사용하면 작은 데이터셋으로 빠르게 반복하여 Amazon SageMaker와 함께 컨테이너를 사용하기 전에 구조적 버그를 제거할 수 있습니다. 테스트는 이 랩의 초점이 아니지만, 시간이 있을 때 예제를 확인해 보세요.
- `decision_trees`는 컨테이너에 설치될 파일들이 포함된 디렉토리입니다.

이 간단한 애플리케이션에서는 컨테이너에 다섯 개의 파일만 설치합니다. 이 다섯 개의 파일은 Python 컨테이너의 표준 구조를 보여주지만, 다른 도구 세트나 프로그래밍 언어를 선택하여 다른 레이아웃을 가질 수도 있습니다.

컨테이너에 넣을 파일들은 다음과 같습니다:

- `nginx.conf`는 nginx 프론트엔드의 구성 파일입니다. 일반적으로 이 파일은 그대로 사용할 수 있습니다.
- `predictor.py`는 Flask 웹 서버와 이 앱의 의사결정 트리 예측을 실제로 구현하는 프로그램입니다. 실제 예측 부분을 애플리케이션에 맞게 사용자 정의해야 합니다. 이 알고리즘은 간단하므로 모든 처리를 이 파일에서 수행하지만, 사용자 정의 로직을 구현하기 위해 별도의 파일을 사용할 수도 있습니다.
- `serve`는 호스팅을 위해 컨테이너가 시작될 때 시작되는 프로그램입니다. 이는 단순히 predictor.py에 정의된 Flask 앱의 여러 인스턴스를 실행하는 gunicorn 서버를 시작합니다. 이 파일은 그대로 사용할 수 있습니다.
- `train`은 훈련을 위해 컨테이너가 실행될 때 호출되는 프로그램입니다. 훈련 알고리즘을 구현하기 위해 이 프로그램을 수정해야 합니다.
- `wsgi.py`는 Flask 앱을 호출하는 데 사용되는 작은 래퍼입니다. 이 파일은 그대로 사용할 수 있습니다.

요약하자면, 애플리케이션을 위해 변경하고 싶을 두 파일은 `train`과 `predictor.py`입니다.

### 패키지 설치
계속 진행하려면 `Python 3 (ipykernel)` 커널을 선택해주세요.

먼저 필요한 패키지들을 설치하겠습니다.

In [ ]:
# cell 00

!pip install --root-user-action=ignore --upgrade pip
!pip install --root-user-action=ignore -q pandas==2.1.4
!pip install --root-user-action=ignore -q awswrangler==3.5.1 --no-cache

### Docker 설치

Docker를 사용하려면 JupyterLab 애플리케이션의 터미널에서 수동으로 설치해야 합니다. Studio에서 현재 지원되는 docker 작업에 대해 숙지하시기 바랍니다 [여기를 참조하세요](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html).

In [ ]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version

그런 다음 필요한 파일들을 압축 해제하고 복사하겠습니다:
- `scikit_bring_your_own/container` → `lab03_container`
- `scikit_bring_your_own/data` → `lab03_data`

In [ ]:
# cell 01

!unzip -q scikit_bring_your_own.zip
!mv scikit_bring_your_own/data/ ./lab03_data/
!mv scikit_bring_your_own/container/ ./lab03_container/
!rm -rf scikit_bring_your_own

### Dockerfile
`Dockerfile`은 우리가 빌드하고자 하는 이미지를 설명합니다. 이것은 실행하고자 하는 시스템의 완전한 운영 체제 설치를 설명하는 것으로 생각할 수 있습니다. 그러나 Docker 컨테이너는 기본 작업을 위해 호스트 머신의 Linux를 활용하기 때문에 전체 운영 체제보다 훨씬 가볍습니다.

Python 과학 스택을 위해, 우리는 표준 Ubuntu 설치에서 시작하여 `scikit-learn`에 필요한 것들을 설치하기 위한 일반적인 도구를 실행할 것입니다. 마지막으로, 우리의 특정 알고리즘을 구현하는 코드를 컨테이너에 추가하고 실행하기 위한 올바른 환경을 설정합니다.

`Dockerfile` 내부를 살펴보겠습니다:

In [ ]:
!pygmentize lab03_container/Dockerfile

### 컨테이너 빌드 및 등록하기

In [ ]:
%%sh
# Login to ECR
aws --region ${AWS_DEFAULT_REGION} ecr get-login-password | docker login --username AWS --password-stdin ${AWS_ACCOUNT_ID}.dkr.ecr.${AWS_DEFAULT_REGION}.amazonaws.com/sagemaker-decision-trees

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "sagemaker-decision-trees" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "sagemaker-decision-trees" > /dev/null
fi

cd lab03_container

chmod +x decision_trees/train
chmod +x decision_trees/serve

# Build the image - it might take a few minutes to complete this step
docker build --network sagemaker . -t ${AWS_ACCOUNT_ID}.dkr.ecr.${AWS_DEFAULT_REGION}.amazonaws.com/sagemaker-decision-trees:latest
# Push the image to ECR
docker push ${AWS_ACCOUNT_ID}.dkr.ecr.${AWS_DEFAULT_REGION}.amazonaws.com/sagemaker-decision-trees:latest

## 설정 및 데이터 업로드

### 환경 설정
여기서는 사용할 버킷과 SageMaker 작업에 사용될 역할을 지정합니다.

In [ ]:
# cell 03

S3_prefix = "DEMO-scikit-byo-iris"

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

SageMaker에 대한 연결 매개변수를 세션이 기억합니다. 모든 SageMaker 작업을 수행하기 위해 이 세션을 사용하겠습니다.

In [ ]:
# cell 04

import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

### S3 버킷에 데이터 업로드하기

대량의 데이터로 대규모 모델을 훈련할 때는 일반적으로 Amazon Athena, AWS Glue 또는 Amazon EMR과 같은 빅 데이터 도구를 사용하여 S3에 데이터를 생성합니다. 이 예제에서는 `lab03_data` 디렉토리에 있는 [고전적인 Iris 데이터셋](https://en.wikipedia.org/wiki/Iris_flower_data_set)을 사용하겠습니다.

[SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/)에서 제공하는 도구를 사용하여 데이터를 기본 버킷에 업로드할 수 있습니다.

In [ ]:
# cell 05

WORK_DIRECTORY = "lab03_data"

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=S3_prefix)

## 모델 훈련

SageMaker를 사용하여 알고리즘을 학습시키기 위해, 컨테이너를 학습에 어떻게 사용할지 정의하는 [`estimator`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)를 생성합니다. 여기에는 SageMaker 학습을 실행하는 데 필요한 다음과 같은 구성이 포함됩니다:

- `image_uri (str)` - Docker 이미지가 등록된 [Amazon Elastic Container Registry](https://aws.amazon.com/ecr/) 경로입니다. 이는 *cell 06*의 셸 명령어에서 구성됩니다.
- `role (str)` - *cell 03*에서 얻은 SageMaker IAM 역할입니다.
- `instance_count (int)` - 학습에 사용할 머신의 수입니다.
- `instance_type (str)` - 학습에 사용할 머신의 유형입니다.
- `output_path (str)` - 모델 아티팩트가 저장될 위치입니다.
- `sagemaker_session (sagemaker.session.Session)` - *cell 04*에서 정의한 SageMaker 세션 객체입니다.

그런 다음 `estimator.fit()` 메서드를 사용하여 업로드한 데이터로 학습을 진행합니다.
이 API는 Amazon SageMaker `CreateTrainingJob` API를 호출하여 모델 학습을 시작합니다. API는 `estimator`를 생성하기 위해 제공한 구성과 지정된 학습 데이터를 사용하여 Amazon SageMaker에 `CreatingTrainingJob` 요청을 보냅니다.

In [ ]:
# cell 06

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image_uri = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest".format(account, region)

tree = sage.estimator.Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
)

file_location = data_location + "/iris.csv"
tree.fit(file_location)

## 모델 배포
HTTP 엔드포인트를 사용하여 학습된 모델로 실시간 예측을 얻을 수 있습니다. 다음 단계에 따라 프로세스를 진행해 보겠습니다.

모델 학습이 성공적으로 완료된 후, [`estimator.deploy()` 메서드](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator.deploy)를 호출할 수 있습니다. `deploy()` 메서드는 배포 가능한 모델을 생성하고, SageMaker 호스팅 서비스 엔드포인트를 구성하며, 모델을 호스팅하기 위한 엔드포인트를 시작합니다.

이 메서드는 다음 구성을 사용합니다:
- `initial_instance_count (int)` – 모델을 배포할 인스턴스 수입니다.
- `instance_type (str)` – 배포된 모델을 운영할 인스턴스 유형입니다.
- `serializer (int)` – 이 예제에서는 다양한 형식(NumPy 배열, 리스트, 파일 또는 버퍼)의 입력 데이터를 CSV 형식의 문자열로 직렬화합니다.

In [ ]:
# cell 07

from sagemaker.serializers import CSVSerializer

predictor = tree.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=CSVSerializer()
)

## 추론 실행하기

### 테스트 데이터 준비
예측을 수행하기 위해, 학습에 사용했던 데이터 중 일부를 추출하여 이에 대한 예측을 수행할 것입니다. 물론 이는 통계적으로 좋은 방법은 아니지만, 메커니즘이 어떻게 작동하는지 쉽게 확인할 수 있는 방법입니다.

In [ ]:
print(file_location)

In [ ]:
# cell 08
import awswrangler as wr

shape = wr.s3.read_csv(file_location, header=None)

# shape=pd.read_csv(file_location, header=None)
shape.sample(3)

In [ ]:
# cell 09

# drop the label column in the training set
shape.drop(shape.columns[[0]], axis=1, inplace=True)
shape.sample(3)

In [ ]:
# cell 10

import itertools

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = shape.iloc[indices[:-1]]

### 예측

예측은 `deploy`에서 반환받은 `predictor`와 예측하려는 데이터로 `predict`를 호출하는 것만큼 간단합니다. 직렬화 도구가 데이터 변환을 대신 처리해 줍니다.

In [ ]:
# cell 11

print(predictor.predict(test_data.values).decode("utf-8"))

## 정리
랩을 완료한 후, [AWS 콘솔을 통해 엔드포인트를 삭제](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html)하거나 다음 코드를 실행하세요.

In [ ]:
# cell 12
sess.delete_endpoint(predictor.endpoint_name)

다운로드한 컨테이너 아티팩트와 데이터를 제거합니다.

In [ ]:
# cell 13
!rm -rf lab03_container lab03_data